# uproot 3 features

This notebook is primarily about what you can do in uproot 3 that you couldn't do in uproot 2. See the [main tutorial](https://mybinder.org/v2/gh/scikit-hep/uproot/master?filepath=binder%2Ftutorial.ipynb) for a basic introduction to uproot.

The main differences are:

   * **more modularization:** uproot is concerned purely with file I/O; functions for interacting with histograms and physics objects like TLorentzVectors have been moved to [uproot-methods](https://github.com/scikit-hep/uproot-methods). Nested data (jagged arrays) have moved to the new [awkward-array](https://github.com/scikit-hep/awkward-array) library. Also, uproot no longer has its own implementation of dict-like caches— instead, we use the third-party [cachetools](https://cachetools.readthedocs.io/en/latest/) library. As a user, this means that you can use whichever combination of versions you need: e.g. bleeding-edge object methods with a stable file I/O.
   * **jagged and object array operations:** now when you read data with structure or class definitions, you can perform Numpy-like operations on them. More on that below.
   * **writing ROOT files:** basic support for writing ROOT files has begun, with more on the way.

In [1]:
import uproot
events = uproot.open("../tests/samples/HZZ-objects.root")["events"]

## object arrays

Since uproot 2, you could read TTree branches describing class objects (defined by ROOT's streamers). This includes STL collections, TVectors, and even histograms— if you should want to store histograms in a TTree. However, it also meant leaving the high-speed Numpy interaction for slow pure-Python objects. In uproot 3, fixed-size objects are interpreted as `awkward.ObjectArrays`, which do most operations in a vectorized way across the fields of the data.

(The same can't be done for variable-sized arrays without custom C code.)

In [2]:
MET = events.array("MET")
MET

<ArrayMethods [TVector2(5.9128, 2.5636) TVector2(24.765, -16.349) TVector2(-25.785, 16.237) ... TVector2(18.102, 50.291), TVector2(79.875, -52.351), TVector2(19.714, -3.5954)] at 7f7d9c1390f0>

The `ObjectArray` above appears to be a list of `TVector2` objects. However, it only creates `TVector2` objects on demand (such as in the print-out). What we have actually loaded is the individual fields (`fX` and `fY`) as separate Numpy arrays.

In [6]:
events["MET"].interpretation

asobj(<uproot_methods.classes.TVector2.Methods>)

In [7]:
events["MET"].interpretation.content

astable(asdtype("[(' fBits', '>u8'), (' fUniqueID', '>u8'), ('fX', '>f8'), ('fY', '>f8')]", "[('fX', '<f8'), ('fY', '<f8')]"))